# Can we identify clothes?

In this exercise we explore a computer vision task called Fashion MNIST. This dataset contains black-and-white images of clothing. The objective is to gain high test accuracy in classifying these images using a convolutional neural network.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

random_gen = np.random.default_rng()
random_numbers = random_gen.integers(low=0, high=x_train.shape[0], size=9)

plt.figure(figsize=(5,5))
for idx, example in enumerate(random_numbers):
    plt.subplot(3, 3, idx+1)
    plt.imshow(x_train[example, :, :], cmap='gray')

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [ ]:
NUM_CONV_LAYERS = 1
KERNEL_SIZE = (3, 3)
NUM_MLP_LAYERS = 1
NUM_MLP_UNITS = 1
NUM_EPOCHS = 5
DROPOUT = 0.1 # usually less than 0.5
REG_COEFFICIENT = 0.001 # Usually less than 0.001


# -- DO NOT TOUCH -- #
x_train_conv = np.expand_dims(x_train, -1)
x_test_conv = np.expand_dims(x_test, -1)


inputs = tf.keras.Input(shape=x_train_conv.shape[1:])
output = inputs
for _ in range(NUM_CONV_LAYERS):
    output = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=KERNEL_SIZE,
        padding='same',
        activation='relu',
    )(output)

output = tf.keras.layers.Flatten(input_shape=x_train.shape[1:])(output)
for _ in range(NUM_MLP_LAYERS):
    output = tf.keras.layers.Dense(
        units=NUM_MLP_UNITS,
        activation='relu'
    )(output)
    output = tf.keras.layers.Dropout(DROPOUT)(output)

output = tf.keras.layers.Dense(
    units=y_test.shape[1],
    activation='softmax'
)(output)
model = tf.keras.Model(inputs, output)
model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

model.summary()
history = model.fit(x_train_conv, y_train, epochs=NUM_EPOCHS)
test_metrics = model.evaluate(x_test_conv, y_test)

plt.figure(figsize=(5, 5))
plt.plot(np.arange(NUM_EPOCHS), np.array(history.history['accuracy']) * 100)
plt.title(
    "Model accuracy v. epochs of training\n"
    f"Train Acc={round(history.history['accuracy'][-1]*100, 4)}%, Test Acc={round(test_metrics[1]*100, 4)}%"
)
plt.xlabel("Epoch")
plt.ylabel("Training accuracy (%)")
